In [1]:

import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[6:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/pgm_sampled'
runs = []
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key',
       'spec.index.leaf_size_in_pages', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')


In [6]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "btree256": 256,
    "btree1024": 1024,
    "btree2048": 2048,
    "sampledpgm256": 256,
    "sampledpgm1024": 1024,
    "sampledpgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def get_epsilon(x):
    for key,eps in epsilon.items():
        if key in x:
            return eps
    return eps

def indexType(x):
    if "sampledpgm" in x:
        return "SAMPLED_PGM"
    if "flatpgm" in x:
        return "FLAT_PGM"
    elif "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    elif "sj" in x:
        return "SORT_JOIN"
    return "NA"


get_epsilon('samm')
df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['epsilon'] = df['spec.algo_name'].map(lambda x: get_epsilon(x))

print(df['join_algo'].unique())


['inlj']


In [9]:
def plot_for_dataset(dataset, join_algo):
    rows = duckdb.sql(
        " SELECT ratio, index_type, epsilon, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND join_algo='{join_algo}'"
        " GROUP BY ratio, join_algo, index_type, epsilon"
        " ORDER BY ratio"
    ).df()
    rows = rows.pivot(index=['ratio', 'epsilon'], values='v', columns=['index_type'])
    return rows


# INLJ (Raw Lookup)

In [10]:
for dataset in ['fb', 'wiki', 'books', 'osm', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal']:
    display(dataset, plot_for_dataset(dataset, 'inlj'))

'fb'

index_type         BTREE        PGM  SAMPLED_PGM
ratio epsilon                                   
1     256      52.997535  39.453556    43.966401
      2048     51.550451  39.072489    41.591839
10    256       6.077259   4.968723     5.841938
      2048      5.644846   5.247334     5.706938
100   256       1.156159   0.988131     1.224114
      2048      1.011372   1.233540     1.439287
1000  256       0.251792   0.263494     0.284928
      2048      0.416693   0.467534     0.496296

'wiki'

index_type         BTREE        PGM  SAMPLED_PGM
ratio epsilon                                   
1     256      23.860882  15.230753    16.742186
      2048     23.257075  16.640457    18.947941
10    256       5.068030   4.119503     4.146453
      2048      5.053400   4.021238     4.764030
100   256       0.871742   0.705935     0.917204
      2048      0.799849   0.901874     1.110890
1000  256       0.228204   0.209108     0.245862
      2048      0.265559   0.321189     0.352403

'books'

...

index_type         BTREE        PGM  SAMPLED_PGM
ratio epsilon                                   
1     256      53.092773  35.211651    35.454066
      2048     51.461066  35.709814    36.702624
10    256       5.989855   4.437733     5.811819
      2048      5.848703   4.660040     4.783330
100   256       1.129902   0.955199     1.189360
      2048      1.007113   1.206858     1.331204
1000  256       0.248898   0.276263     0.296285
      2048      0.439503   0.448134     0.487433

'normal'

index_type         BTREE        PGM  SAMPLED_PGM
ratio epsilon                                   
1     256      53.029567  29.799271    33.330135
      2048     51.502682  33.366232    37.709050
10    256       6.088343   3.915963     5.282751
      2048      5.886218   4.384128     5.682568
100   256       1.165795   1.413126     1.186780
      2048      1.017993   1.272267     1.485959
1000  256       0.253460   0.274309     0.284768
      2048      0.441938   0.463282     0.491317

'lognormal'

index_type         BTREE        PGM  SAMPLED_PGM
ratio epsilon                                   
1     256      53.162460  31.239963    34.989338
      2048     51.491722  33.789932    37.503160
10    256       6.090505   4.182839     5.568489
      2048      5.671901   4.952037     6.258265
100   256       1.155672   0.984703     1.276678
      2048      1.028822   1.480253     1.758271
1000  256       0.238581   0.266859     0.315938
      2048      0.422554   0.515832     0.521484